In [1]:
import pandas as pd
import numpy as np
import multiprocessing as mp
cores_to_leave = 2
search = True # Boolean for if we run a random search
run = False # Boolean for model determining runs

In [2]:
housing = pd.read_csv('./housing.csv')

# Preprocessing

In [3]:
# Split into test and train sets
from sklearn.model_selection import train_test_split
train, test = train_test_split(housing.copy(), test_size=0.2) # Works on dataframes
y_train = train['median_house_value'].values # Train targets
y_test = test['median_house_value'].values # Test targets
train.drop(['median_house_value'], axis=1, inplace=True) # Train: Leave only features
test.drop(['median_house_value'], axis=1, inplace=True) # Test: Leave only features
train.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
4339,-118.29,34.08,34.0,479.0,182.0,557.0,170.0,1.5250,<1H OCEAN
236,-122.20,37.79,49.0,882.0,195.0,737.0,210.0,2.6667,NEAR BAY
20564,-121.71,38.72,32.0,710.0,155.0,550.0,154.0,2.8882,INLAND
8369,-118.34,33.98,47.0,2649.0,684.0,2374.0,607.0,2.3882,<1H OCEAN
13253,-117.68,34.11,16.0,3190.0,471.0,1414.0,464.0,5.5292,INLAND


In [4]:
# Make an object for selecting specific variables of a DataFrame
# Will be useful for building pipelines later as we can just input a dataframe
from sklearn.base import BaseEstimator, TransformerMixin

# Class MUST have a fit and transform function as this is what functions sklearn will look for in the pipeline
# Thus we must watch the syntax
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attributes): # attributes represents a list of columns you want to select
        self.attributes = attributes 
    # fit: This would be any transformations you want to perform (in this case we are just selecting columns)
    # Note that you must include the y=None to match sklearn syntax
    def fit(self, X, y=None): 
        return self
    # transform: return selected columns
    def transform(self, X): # X is a dataframe input
        return X[self.attributes] # Return selected values

In [5]:
# Make another function for adding specific attributes
# Currently just replacing Longitude, Lattitude w/ Spherical Coordinates
class GeoReplacer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X): # X is a dataframe input
        X['sphere_x'] = np.cos(np.radians(X['latitude'])) * np.cos(np.radians(X['longitude']))
        X['sphere_y'] = np.cos(np.radians(X['latitude'])) * np.sin(np.radians(X['longitude']))
        X['sphere_z'] = np.sin(np.radians(X['latitude']))
        X.drop(columns=['latitude', 'longitude'], axis=1, inplace=True) # Drop these as they are unwanted
        return X.values

In [6]:
# First step, we want to impute the missing categorical values
# NOTE: must use train data ONLY to impute, don't want to touch test data
from sklearn.impute import SimpleImputer
numeric_cols = train.select_dtypes([np.number]).columns.tolist() # Get a list of numeric columns
selector_num = DataFrameSelector(numeric_cols) # Object for selecting numeric columns
train_impute = selector_num.transform(train) # Transform the data
imputer = SimpleImputer(missing_values=np.nan, strategy='median') # Make the imputer - will use on test data later
imputer.fit(train_impute) # Fit the data
X_impute = imputer.transform(train_impute) # Transform the data

In [7]:
# One hot encode the categorical data
from sklearn.preprocessing import OneHotEncoder
categorical_cols = train.select_dtypes(['object']).columns.tolist() # Categorical columns
selector_cat = DataFrameSelector(categorical_cols) # Initialize selector
train_onehot = selector_cat.transform(train) # Transform the data
onehot = OneHotEncoder() # Generate OneHotEncoder object
onehot.fit(train_onehot) # Fit
X_onehot = onehot.transform(train_onehot) # Transform the data

In [8]:
# Standard scaler on numerical data
# Note: Don't include targets here!
# Per: https://stackoverflow.com/questions/26584971/how-to-not-standarize-target-data-in-scikit-learn-regression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
x_scaler = StandardScaler() # initialize object
x_scaler.fit(X_impute) # Fit imputed data
X_scaled = x_scaler.transform(X_impute)

# Pipeline Structure
* I think all of the above can be done in a pipeline structure
* Basically performing all these steps in a defined sequence
* Good to document it out once though just for practice

In [9]:
# Create preprocessing pipelines
from sklearn.pipeline import Pipeline, FeatureUnion

# Numerical feature pipe
numerical_pipe = Pipeline([
    ('num_select', DataFrameSelector(numeric_cols)), # SELECT COLUMNS FIRST
    ('geo_replace', GeoReplacer()), # THEN TRANSFORM
    ('imputer', SimpleImputer()),
    ('scaler', StandardScaler())
])

# Categorical feature pipe
categorical_pipe = Pipeline([
    ('cat_select', DataFrameSelector(categorical_cols)),
    ('onehot', OneHotEncoder())
])

# Combine into pre-processing pipeline single object
prep_pipe = FeatureUnion([
    ('numerical', numerical_pipe),
    ('categorical', categorical_pipe)
])

# NOTE: Can access elements of a feature union for random/grid search per https://www.kaggle.com/edolatabadi/feature-union-with-grid-search
# Can then run everything through these at once!

# Model Selection
* Select sample models
* Create a scoring object 
* Evaluate and see which models perform best

In [10]:
# Models to test
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
models = {'RandomForest':RandomForestRegressor(),
         'AdaBoost':AdaBoostRegressor(),
         'GradBoost':GradientBoostingRegressor()}

In [11]:
# Fit pipeline on training data
prep_pipe.fit(train, y_train)
X_train = prep_pipe.transform(train)

In [12]:
# Iterate over models
from sklearn.model_selection import cross_val_score
run = False # Boolean for this cell

# Function for printing out scores
def print_scores(score_array):
    print('Errors: ', score_array)
    print('Mean Error: ', score_array.mean())
    print('Std: ', score_array.std())
    
if run:
    for k, model in models.items():
        print('Model: ' + k)
        m_scores = cross_val_score(model, X_train, y_train, # Have to flatten transformed y_data
                                  scoring='neg_mean_squared_error', cv=10)
        m_scores = np.sqrt(-m_scores) # Because using negative mean squared error
        print_scores(m_scores)

# Randomized Search
* RandomForest was the best performing
* Let's use a randomized search to find best parameters

In [13]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
# Need to make a model pipeline
full_pipe = Pipeline([
    ('prep', prep_pipe),
    ('reg', RandomForestRegressor())
])
full_pipe

Pipeline(steps=[('prep',
                 FeatureUnion(transformer_list=[('numerical',
                                                 Pipeline(steps=[('num_select',
                                                                  DataFrameSelector(attributes=['longitude',
                                                                                                'latitude',
                                                                                                'housing_median_age',
                                                                                                'total_rooms',
                                                                                                'total_bedrooms',
                                                                                                'population',
                                                                                                'households',
                                                     

In [14]:
# Set param grid
param_grid = {
    'prep__numerical__scaler':[StandardScaler(), MinMaxScaler()],
    'reg__bootstrap':[True, False],
    'reg__n_estimators':[10, 50, 100, 200, 300],
    'reg__max_features':[2, 4, 6, 8, 10]
}

if search:
    # Run Search
    searcher = GridSearchCV(full_pipe, param_grid, n_jobs=5) # Default 5 fold for grid search
    search_res = searcher.fit(train, y_train)

In [15]:
# What are our best parameters?
print('Best Params: ', search_res.best_params_)
all_results = pd.DataFrame.from_dict(search_res.cv_results_) # Dataframe of all results
all_results.to_csv('./grid_search_results')
print(all_results.head())

# Set best model params
# Note: could also use search_res.best_estimator_ to get the best performing model
full_pipe = full_pipe.set_params(**search_res.best_params_) # Set pipeline with best parameters

Best Params:  {'prep__numerical__scaler': MinMaxScaler(), 'reg__bootstrap': False, 'reg__max_features': 4, 'reg__n_estimators': 200}
   mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0       1.263146      0.026284         0.014213        0.000400   
1       6.239221      0.054300         0.045441        0.002061   
2      11.565435      0.212375         0.081074        0.004821   
3      23.900326      0.195370         0.151738        0.002061   
4      34.106656      0.296401         0.223003        0.002318   

  param_prep__numerical__scaler param_reg__bootstrap param_reg__max_features  \
0              StandardScaler()                 True                       2   
1              StandardScaler()                 True                       2   
2              StandardScaler()                 True                       2   
3              StandardScaler()                 True                       2   
4              StandardScaler()                 True            

In [16]:
# Now, let's do another cross val with out best model
scores = cross_val_score(full_pipe, train, y_train, # Have to flatten transformed y_data
                          scoring='neg_mean_squared_error', cv=10)
scores = np.sqrt(-scores)
print_scores(scores)

Errors:  [46375.86781174 44543.90994171 47143.05195741 48231.20909617
 44763.89551764 44519.99821026 45656.16170849 44802.01564379
 48489.10531814 43696.20205432]
Mean Error:  45822.141725967434
Std:  1581.2932099460072
